In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
#root_path = 'C:/Users/jsusn/Documents/GitHub/lighthouse-data-notes/final_project/NBA-predictions/data'

from google.colab import drive
drive.mount('/content/drive')

root_path = 'drive/MyDrive/NBA-predictions/data/'

In [ ]:
def plot_confidence(y_test, y_preds, y_probs, bins=30, graph_range=(-1,0)):
    '''
    Plots predict_proba as a histogram.

    Note: The prediction probability is for HOME_WIN = 1, therefore the ideal graph would have most of its data centered around 40-60%.
    a 20% incorrect prediction for the HOME_WIN = 1 (when it should be 0) is the same magnitude of error as an 80% for predicting HOME_WIN = 0 (when it should be 1)

    '''
    import seaborn as sns
    import matplotlib.pyplot as plt
    sns.set(rc={'figure.figsize':(20, 8)})

    hist_data = pd.DataFrame()
    hist_data['TRUTH'] = y_test
    hist_data['PROBS'] = y_probs
    hist_data['PREDS'] = y_preds

    graph_col = list()
    for row in range(len(hist_data)):
        if hist_data.iloc[row].TRUTH == hist_data.iloc[row].PREDS: # the TRUTH matches the PREDICTION, append a POSITIVE PROBS
            graph_col.append(hist_data.iloc[row].PROBS)
        else:
            graph_col.append(hist_data.iloc[row].PROBS * (-1)) # the TRUTH does NOT match the PREDICTION, append a NEGATIVE PROBS

    hist_data['GRAPH'] = graph_col

    plt.hist([hist_data['GRAPH']], color=['blue'], bins=bins, range=graph_range)

    df_filter = hist_data.GRAPH < 0
    print('Mean confidence of error: ' + str(hist_data[df_filter]['GRAPH'].mean()))
    total_incorrect = hist_data[df_filter]['GRAPH'].count()

    df_filter = (hist_data.GRAPH < (-0.65)) | ((hist_data.GRAPH > (-0.35)) & (hist_data.GRAPH < (0)))
    print('Number of "strongly" incorrect (greater than 65% probability): ' + str(hist_data[df_filter]['GRAPH'].count()) + ' of ' + str(total_incorrect) + ' total incorrect.')

    return

In [ ]:
train_data = pd.read_csv(root_path +'/train_data_ppmadf.csv')
test_data= pd.read_csv(root_path +'/test_data_ppmadf.csv')

X_train = train_data.drop(columns=['HOME_WIN'])
y_train = train_data['HOME_WIN']

X_test = test_data.drop(columns=['HOME_WIN'])
y_test = test_data['HOME_WIN']

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.15, max_depth = 6, reg_alpha = 0.0001, n_estimators = 1000, reg_lambda = 10, random_state = 101)
xg_reg.fit(X_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.15, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=101, reg_alpha=0.0001, reg_lambda=10,
             scale_pos_weight=1, seed=None, silent=None, subsample=1,
             verbosity=1)

In [ ]:
preds = xg_reg.predict(X_test)
preds = np.round(preds,0)
preds

array([1., 1., 0., ..., 1., 1., 1.], dtype=float32)

In [ ]:
print(accuracy_score(y_test, preds))

0.6382113821138211


In [ ]:
xg_clf = xgb.XGBClassifier(objective ='binary:logistic', learning_rate = 0.15, max_depth = 6, reg_alpha = 0.0001, n_estimators = 750, reg_lambda = 5, random_state = 101, enable_categorical = True)
xg_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=True,
              gamma=0, learning_rate=0.15, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=750, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=101,
              reg_alpha=0.0001, reg_lambda=5, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
preds = xg_clf.predict(X_test)
print(accuracy_score(y_test, preds))

0.6357723577235772
